<img alt="QuantRocket logo" src="https://www.quantrocket.com/assets/img/notebook-header-logo.png">

<a href="https://www.quantrocket.com/disclaimer/">Disclaimer</a>

# Migrate Databases

In version 1, all databases begin with `quantrocket.*`, while in version 2, all databases begin with `quantrocket.v2.*`. Thus, version 1 and version 2 databases can exist side-by-side. 

To populate version 2 databases, migration scripts are provided for the master database, the blotter databases, the account balances database, and selected fundamentals databases. For other databases, migration scripts are not provided. Instead, please re-collect any data you wish to use in version 2. 

Why are there migration scripts for some databases and not others? We have focused on providing migration scripts for databases containing historical records that can't be otherwise recovered.

> NOTE: The migration scripts copy data from your version 1 databases to your version 2 databases. No changes are made to your version 1 databases.

## Migrate the master database

Migrating the master database involves moving the records from the v1 database to the v2 database and assigning the correct sids.

To do this, first collect one or more IBKR securities using the v2 API. This will trigger QuantRocket to load the complete mapping of IBKR conids to sids. The following command which simply loads EUR.USD can be used for this step. (You need not try to collect every listing in your v1 master database.)

In [1]:
from quantrocket.ibg import start_gateways
start_gateways(wait=True)

{'ibg1': {'status': 'running'}}

In [2]:
from quantrocket.master import collect_ibkr_listings
collect_ibkr_listings(exchanges="IDEALPRO", symbols="EUR.USD", sec_types="CASH")

{'status': 'the IBKR listing details will be collected asynchronously'}

Monitor flightlog for completion:

```
quantrocket.master: INFO Collecting IDEALPRO CASH listings from IBKR website (EUR.USD only)
quantrocket.master: INFO Requesting details for 1 IDEALPRO listings found on IBKR website
quantrocket.master: INFO Saved 1 IDEALPRO listings to securities master database
quantrocket.master: INFO Building consolidated securities master from active vendors: ibkr
quantrocket.master: INFO Completed building consolidated securities master with 1 records
```

Now you are ready to migrate the master database:

In [3]:
from codeload.migrate_v1_to_v2.migratedb import migrate_master
migrate_master()

copied v1 master to v2


The data is now loaded into various IBKR tables in the securities master database, but now QuantRocket needs to rebuild the consolidated master table so the records will be available via the API. To trigger this process, simply collect any IBKR listing again:

In [4]:
collect_ibkr_listings(exchanges="IDEALPRO", symbols="EUR.USD", sec_types="CASH")

{'status': 'the IBKR listing details will be collected asynchronously'}

Monitor flightlog for completion. This time you should see that more records are included in the consolidated master, reflecting the number of migrated listings:

```
...
quantrocket.master: INFO Completed building consolidated securities master with 42990 records
```

You can now query some of the migrated data: 

In [5]:
!quantrocket master get -s 'AAPL' -e 'NASDAQ' -j | json2yml

---
  - 
    Sid: "FIBBG000B9XRY4"
    Symbol: "AAPL"
    Exchange: "XNAS"
    Currency: "USD"
    SecType: "STK"
    Etf: 0
    Timezone: "America/New_York"
    Name: "APPLE INC"
    PriceMagnifier: 1
    Multiplier: 1
    Delisted: null
    DateDelisted: null
    LastTradeDate: null
    RolloverDate: null



By default core fields are returned in version 2. Additional IBKR fields are still available on request: 

In [6]:
!quantrocket master get -s 'AAPL' -e 'NASDAQ' --fields 'ibkr*' -j | json2yml

---
  - 
    Sid: "FIBBG000B9XRY4"
    ibkr_AggGroup: 1
    ibkr_Category: "Computers"
    ibkr_ComboLegs: null
    ibkr_ConId: 265598
    ibkr_ContractMonth: null
    ibkr_Currency: "USD"
    ibkr_Cusip: null
    ibkr_DateDelisted: null
    ibkr_Delisted: 0
    ibkr_Etf: 0
    ibkr_EvMultiplier: 0
    ibkr_EvRule: null
    ibkr_Industry: "Computers"
    ibkr_Isin: null
    ibkr_LastTradeDate: null
    ibkr_LocalSymbol: "AAPL"
    ibkr_LongName: "APPLE INC"
    ibkr_MarketName: "NMS"
    ibkr_MarketRuleIds: "26,26,26,26,26,26,26,26,26,26,26,26,26"
    ibkr_MdSizeMultiplier: 100
    ibkr_MinTick: 0.01
    ibkr_Multiplier: null
    ibkr_PriceMagnifier: 1
    ibkr_PrimaryExchange: "NASDAQ"
    ibkr_RealExpirationDate: null
    ibkr_Right: null
    ibkr_SecType: "STK"
    ibkr_Sector: "Technology"
    ibkr_Strike: 0
    ibkr_Symbol: "AAPL"
    ibkr_Timezone: "America/New_York"
    ibkr_TradingClass: "NMS"
    ibkr_UnderConId: 0
    ibkr_UnderSecType: null
    ibkr_UnderSymbol: null
    ibk

#### Notice to Sharadar subscribers

The above command migrates the data in `quantrocket.master.main.sqlite` only. There is no separate Sharadar master in version 2. If you are a Sharadar susbcriber and created universes in `quantrocket.master.sharadar.sqlite`, please follow the usage guide to re-collect Sharadar listings, then re-create any Sharadar universes you may require. 

## Migrate the blotter and account databases

Next, migrate the blotter databases:


In [7]:
from codeload.migrate_v1_to_v2.migratedb import migrate_blotter
migrate_blotter()

copied v1 orders to v2
copied v1 order errors to v2
copied v1 order statuses to v2
copied v1 executions and positions to v2


And the account balance database:

In [8]:
from codeload.migrate_v1_to_v2.migratedb import migrate_account_balances
migrate_account_balances()

copied v1 account balances to v2


NOTE: the exchange rates database (`quantrocket.v2.account.rates.sqlite`) will automatically collect historical exchange rates. The PNL database (`quantrocket.v2.blotter.pnl.sqlite`) will be automatically generated from the executions database. No action is required.

## Fundamental Data

For fundamental data, migration scripts are provided for the Reuters estimates database and the Wall Street Horizon database. Both of these databases provide a rolling window of data or forward-looking data, which means that without a migration script some history would be lost. This is also true of the Reuters financials database, however we have not provided a migration script for the Reuters financials database due to the additional complexity of table sharding used in the database. 

For other databases (Sharadar, IBKR shortable shares, IBKR borrow fees), please re-collect the data as needed. 

### Reuters estimates


In [9]:
from codeload.migrate_v1_to_v2.migratedb import migrate_reuters_estimates
migrate_reuters_estimates()

copied v1 Reuters estimates to v2


### Wall Street Horizon Earnings Calendar

In [10]:
from codeload.migrate_v1_to_v2.migratedb import migrate_wsh_calendar
migrate_wsh_calendar()

copied v1 WSH calendar to v2


## Historical and real-time data

We have not provided migration scripts for historical databases or real-time databases. For historical data, please follow the usage guide to re-collect any historical data you wish to use in version 2. For real-time databases, please create new databases and begin collecting real-time data anew in version 2.

We understand that real-time data cannot be backfilled in this manner and also that historical data for certain expired futures and options contracts you collected in v1 may no longer be available from IBKR. We apologize for the inconvenience of not carrying this data over to version 2. The numerous sharding options used by historical databases, and the complex TimescaleDB database structure used by real-time databases, would have made migration of these databases more complex.

## Purging version 1 databases

Later, if you are happy with the version 2 migration and would like to get rid of old version 1 databases to free up disk space, please do so from a terminal, as the version 2 software will not "see" the databases and thus they cannot be deleted via the API. For example, to delete the old exchange rates database:

```shell
rm /var/lib/quantrocket/quantrocket.account.rates.sqlite
```

NOTE: for sharded databases, which utilize a directory structure, you must use `rm -r` to recursively delete the directory. Be careful with `rm -r`!

***

## *Next Up*

Part 2: [Make Code Changes](Part2-Code-Changes.ipynb)